## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [11]:
# Let's take a look
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 600,165 (2.29 MB)

 Trainable params: 600,165 (2.29 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 51s 213ms/step - accuracy: 0.2030 - loss: 1.6168 - val_accuracy: 0.2107 - val_loss: 1.6023
Epoch 2/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 45s 195ms/step - accuracy: 0.2424 - loss: 1.5995 - val_accuracy: 0.2822 - val_loss: 1.5828
Epoch 3/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 82s 197ms/step - accuracy: 0.2912 - loss: 1.5836 - val_accuracy: 0.4717 - val_loss: 1.5622
Epoch 4/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 47s 203ms/step - accuracy: 0.3635 - loss: 1.5612 - val_accuracy: 0.6143 - val_loss: 1.5393
Epoch 5/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 81s 197ms/step - accuracy: 0.4240 - loss: 1.5411 - val_accuracy: 0.6832 - val_loss: 1.5132
Training time: 0:05:42.211266
Test score: 1.5132181644439697
Test accuracy: 0.6831927299499512


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,800,497 (6.87 MB)

 Trainable params: 590,597 (2.25 MB)

 Non-trainable params: 9,568 (37.38 KB)

 Optimizer params: 1,200,332 (4.58 MB)

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 19s 75ms/step - accuracy: 0.1968 - loss: 1.6034 - val_accuracy: 0.3734 - val_loss: 1.5716
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 18s 74ms/step - accuracy: 0.2892 - loss: 1.5742 - val_accuracy: 0.4933 - val_loss: 1.5416
Epoch 3/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - accuracy: 0.3711 - loss: 1.5469 - val_accuracy: 0.5672 - val_loss: 1.5111
Epoch 4/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.4402 - loss: 1.5181 - val_accuracy: 0.6495 - val_loss: 1.4795
Epoch 5/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 22s 74ms/step - accuracy: 0.5152 - loss: 1.4874 - val_accuracy: 0.7266 - val_loss: 1.4476
Training time: 0:01:41.015497
Test score: 1.4475958347320557
Test accuracy: 0.7266005277633667


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster.

## Exercise
### To do:
- Now write code to reverse this training process.  That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [16]:
# We create our model by combining the two sets of layers as follows
model_2 = Sequential(feature_layers + classification_layers)

In [17]:
train_model(model_2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - accuracy: 0.5583 - loss: 1.4595 - val_accuracy: 0.7708 - val_loss: 1.4164
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.6031 - loss: 1.4282 - val_accuracy: 0.8112 - val_loss: 1.3846
Epoch 3/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 21s 73ms/step - accuracy: 0.6443 - loss: 1.4008 - val_accuracy: 0.8362 - val_loss: 1.3532
Epoch 4/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - accuracy: 0.6830 - loss: 1.3696 - val_accuracy: 0.8543 - val_loss: 1.3217
Epoch 5/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - accuracy: 0.7073 - loss: 1.3420 - val_accuracy: 0.8648 - val_loss: 1.2906
Training time: 0:01:41.711970
Test score: 1.2905715703964233
Test accuracy: 0.8647596836090088


In [18]:
# Freeze only the
for l in feature_layers:
    l.trainable = False

In [19]:
train_model(model_2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 18s 75ms/step - accuracy: 0.4402 - loss: 1.4972 - val_accuracy: 0.5639 - val_loss: 1.4621
Epoch 2/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 21s 76ms/step - accuracy: 0.4828 - loss: 1.4685 - val_accuracy: 0.6050 - val_loss: 1.4350
Epoch 3/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step - accuracy: 0.5085 - loss: 1.4464 - val_accuracy: 0.6453 - val_loss: 1.4084
Epoch 4/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - accuracy: 0.5498 - loss: 1.4201 - val_accuracy: 0.6822 - val_loss: 1.3823
Epoch 5/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 76ms/step - accuracy: 0.5784 - loss: 1.3976 - val_accuracy: 0.7157 - val_loss: 1.3568
Training time: 0:01:34.208558
Test score: 1.356813907623291
Test accuracy: 0.7156963348388672
